In [1]:
import tkinter as tk
from tkinter import Canvas
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
import random
from keras.models import load_model

In [3]:
Path_Data = './Dataset/'

def load_images():
    image_list = [] #이미지 리스트
    label_list = [] #클래스 리트트
    filename_list = [] #파일이름 리스트
    
    for imgclass in os.listdir(Path_Data):  #디렉토리안에 있는 모든 파일을 순서대로 읽음
        if imgclass.endswith(('png','jpg')): #사진일때 True
            img_path = os.path.join(Path_Data,imgclass)
            img = cv2.imread(img_path)
            if img is None: #이미지가 없을때
                continue
            img = cv2.resize(img, (300,300))
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = img/255.0
            label = 1 if 'o' in imgclass.lower() else 0
            image_list.append(img)
            label_list.append(label)
            filename_list.append(imgclass)
            
    return np.array(image_list,dtype=np.float32), np.array(label_list,dtype=np.int32),filename_list

x,y,filenames=load_images()
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=None)
x_test,x_eval,y_test,y_eval = train_test_split(x_test,y_test,test_size=10/len(x_test),random_state=None)

In [8]:
# 모델 파일 경로
model_path = "models_loocv.h5"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file '{model_path}' not found. Please check the file path.")

# 모델 로드
model = tf.keras.models.load_model(model_path)

# tkinter 윈도우 설정
root = tk.Tk()
root.title("O/X Classifier")

canvas_size = 500
canvas = Canvas(root, width=canvas_size, height=canvas_size, bg="white")
canvas.grid(row=0, column=0, columnspan=2)

# 그림 초기화 함수
def clear_canvas():
    canvas.delete("all")

# 캔버스를 이미지로 저장하는 함수
def save_canvas_as_image(canvas, filename="canvas_image.png"):
    # 캔버스를 새로운 이미지로 변환
    img = Image.new("RGB", (canvas_size, canvas_size), "white")
    draw = ImageDraw.Draw(img)

    # 캔버스 위에 그려진 모든 도형 가져오기
    for item in canvas.find_all():
        x1, y1, x2, y2 = canvas.coords(item)
        draw.ellipse([x1, y1, x2, y2], fill="black", outline="black")

    # 저장
    img.save(filename)

# 결과예측 함수
def prediction(model, canvas, save_path="canvas_image.png"):
    save_canvas_as_image(canvas, save_path)  # 캔버스 내용을 PNG로 저장

    # 이미지 전처리
    img = Image.open(save_path)
    img = img.convert("RGB")
    img = ImageOps.invert(img)  # 색상 반전 (검정 바탕 → 흰 바탕)
    img = img.resize((300, 300))  # 모델 입력 크기에 맞게 리사이징
    img = np.array(img) / 255.0  # 정규화
    img = img.reshape(1, 300, 300, 3)  # 모델 입력 차원 맞추기

    # 모델 예측
    predictions = model.predict(img)
    result = "O" if predictions[0][0] >= 0.5 else "X"

    # 결과 표시
    result_label.config(text=f"결과: {result}")

    print(f"Prediction Score: {predictions[0][0]:.4f} → Result: {result}")
    return result

# 마우스 드래그로 그림 그리기
def paint(event):
    x, y = event.x, event.y
    radius = 5
    canvas.create_oval(x - radius, y - radius, x + radius, y + radius, fill="black", outline="black")

# UI 버튼
clear_button = tk.Button(root, text="다시 그리기", command=clear_canvas)
clear_button.grid(row=1, column=0)

eval_button = tk.Button(root, text="평가", command=lambda: prediction(model, canvas))  # 함수 실행 시 인자 전달
eval_button.grid(row=1, column=1)

# 결과 표시 라벨
result_label = tk.Label(root, text="결과: ", font=("Arial", 20))
result_label.grid(row=2, column=0, columnspan=2)

# 캔버스에 그림 그리기 이벤트 바인딩
canvas.bind("<B1-Motion>", paint)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Prediction Score: 1.0000 → Result: O


In [89]:
# from PIL import Image, ImageOps
# import numpy as np

# def prediction(model, canvas, save_path="canvas_image.png"):
#     # 캔버스 내용을 바로 PNG로 저장
#     canvas.update()  # 캔버스 업데이트 (필요할 경우)
#     canvas.postscript(file="canvas_image.eps")  # 임시 .eps 파일 저장
#     img = Image.open("canvas_image.eps")  # .eps 파일 열기
    
#     # PNG로 저장 (eps를 거치지 않으려면 직접 캡처 방식 사용)
#     img = img.convert("RGB")  
#     img.save(save_path, format="PNG")  # PNG 또는 JPG 저장 가능

#     # 이미지 전처리
#     img = Image.open(save_path)
#     img = img.convert("RGB")
#     img = ImageOps.invert(img)  # 색상 반전 (검정 바탕 -> 흰 바탕)
#     img = img.resize((300, 300))  # 모델 입력 크기에 맞게 리사이징
#     img = np.array(img) / 255.0  # 정규화
#     img = img.reshape(1, 300, 300, 3)  # 모델 입력 차원 맞추기

#     # 모델 예측
#     predictions = model.predict(img)
#     result = "O" if predictions[0][0] >= 0.5 else "X"

#     print(f"Prediction Score: {predictions[0][0]:.4f} → Result: {result}")
#     return result


In [85]:
# # 모델 파일 경로
# model_path = 'models_loocv.h5'
# # 모델 파일 존재 여부 확인
# if not os.path.exists(model_path):
#     raise FileNotFoundError(f"Model file '{model_path}' not found. Please check the file path.")
# # 모델 로드
# model = tf.keras.models.load_model(model_path)
# # 이미지 파일 경로
# img_path = r"C:\Users\JongBeom\Desktop\Univ\LAB_study\DEEPLEARNING\Dataset\O\o_1.png"
# # 이미지 로드 및 전처리
# img = image.load_img(img_path, target_size=(300, 300))  # 모델 입력 크기에 맞춰 조정 (ex: 224x224)
# img_array = image.img_to_array(img)  # PIL 이미지 → NumPy 배열 변환
# img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가 (1, 224, 224, 3)
# img_array = img_array / 255.0  # 정규화 (0~1 사이 값)
# # 예측 결과 처리
# predicted_value = predictions[0][0]  # 첫 번째 샘플의 예측값 (모델 출력이 단일 숫자라고 가정)
# result = "O" if predicted_value >= 0.5 else "X"

# # 결과 출력
# print(f"Prediction Score: {predicted_value:.4f} → Result: {result}")

Prediction Score: 1.0000 → Result: O
